In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
# 1. 获取数据
facebook = pd.read_csv('./data/FBlocation/train.csv')

In [3]:
facebook.shape

(29118021, 6)

In [4]:
# 2. 数据基本处理
#    2.1 缩小数据范围
data = facebook.query('x>2 & x<2.5 & y>2 &y<2.5')

In [5]:
data.shape

(71664, 6)

In [6]:
#    2.2 时间转化星期,小时,天
data.head()

,row_id,x,y,accuracy,time,place_id
163,163,2.1663,2.3755,84,669737,3869813743
310,310,2.3695,2.2034,3,234719,2636621520
658,658,2.3236,2.1768,66,502343,7877745055
1368,1368,2.2613,2.3392,73,319822,9775192577
1627,1627,2.3331,2.0011,66,595084,6731326909


In [9]:
time = pd.to_datetime(data['time'],unit='s')   # 脱敏

In [10]:
time = pd.DatetimeIndex(time)

In [12]:
data['weekday'] = time.weekday

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
data['hour'] = time.hour

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
data['day'] = time.day

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
data.head()

,row_id,x,y,accuracy,time,place_id,weekday,hour,day
163,163,2.1663,2.3755,84,669737,3869813743,3,18,8
310,310,2.3695,2.2034,3,234719,2636621520,5,17,3
658,658,2.3236,2.1768,66,502343,7877745055,1,19,6
1368,1368,2.2613,2.3392,73,319822,9775192577,6,16,4
1627,1627,2.3331,2.0011,66,595084,6731326909,2,21,7


In [21]:
#    2.3 删除到的次数少的位置数据
place_count = data.groupby("place_id").count()['x']

In [24]:
place_count = place_count[place_count>3]

In [28]:
data = data[data['place_id'].isin(place_count.index)]

In [29]:
data.shape

(69264, 9)

In [32]:
#    2.4 确定特征值和目标值
x = data[["x",'y','accuracy','weekday','hour','day']]
y = data['place_id']

In [34]:
#    2.5 切割数据
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=22)

In [37]:
# 3. 特征工程
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [38]:
# 4. 建立模型
estimator = KNeighborsClassifier(n_neighbors=3)
param = {"n_neighbors":[1,3,5,7]}
estimator = GridSearchCV(estimator,cv=4,param_grid=param)

In [39]:
estimator.fit(x_train,y_train)

# 5. 模型评估

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=4, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 3, 5, 7]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [40]:
# 5. 模型评估
estimator.score(x_test,y_test)

0.36952284703674293

In [41]:
estimator.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [42]:
estimator.best_score_

0.3564635180740286

In [43]:
estimator.cv_results_

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([0.07987183, 0.08944523, 0.08603328, 0.07332605]),
 'std_fit_time': array([0.00744606, 0.01109834, 0.0202621 , 0.0075271 ]),
 'mean_score_time': array([0.4206605 , 0.7906242 , 0.85856348, 0.94732475]),
 'std_score_time': array([0.05663047, 0.06300705, 0.04221439, 0.02543064]),
 'param_n_neighbors': masked_array(data=[1, 3, 5, 7],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 1},
  {'n_neighbors': 3},
  {'n_neighbors': 5},
  {'n_neighbors': 7}],
 'split0_test_score': array([0.35178395, 0.33754054, 0.34713016, 0.3455789 ]),
 'split1_test_score': array([0.35683608, 0.3480315 , 0.352398  , 0.35354331]),
 'split2_test_score': array([0.35879529, 0.34315437, 0.3558126 , 0.35573985]),
 'split3_test_score': array([0.35861763, 0.34751721, 0.36157774, 0.3578776 ]),
 'mean_test_score': array([0.35646352, 0.34401112, 0.35413546, 0.35310678]),
 'std_test_score': array([0.00284958, 0.0042392 , 0

In [45]:
pre = estimator.predict(x_test)

In [47]:
print("预测值与真实值\n",pre == y_test)

预测值与真实值
 24703810     True
19445902    False
18490063     True
7762709     False
6505956     False
14457513    False
17814238     True
14306056     True
16488083    False
17062930    False
11317096    False
3051666     False
14849232     True
9015944      True
11106006    False
7885852     False
2448673     False
16491341    False
8969522     False
18463134    False
4228061     False
4073153     False
5366876     False
26617983     True
28140899    False
3971601     False
15668533     True
24483192    False
2337316     False
21035784    False
            ...  
14061315    False
26691044    False
14461756     True
25917365    False
4701918     False
991627      False
26137214    False
19034521     True
10852558    False
14884877    False
12699358    False
28587664    False
2009434      True
22970622    False
27543156    False
1432189      True
17870841    False
11486816     True
6150132      True
4021004     False
29005319    False
11697331    False
12965998    False
672605      False
1